In [54]:
import pandas as pd
import numpy as np
import sklearn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

In [9]:
raw_data = pd.read_csv("Data/train.csv")
raw_data.head()

fNames = pd.read_csv("Data/female_names.csv")
fNames=fNames['name']
mNames = pd.read_csv("Data/male_names.csv")
mNames=mNames['name']



True

In [15]:
df=raw_data.copy()
# Converting Booleans to numerical values
df["Transported"] = df["Transported"].astype(int)
df.replace({False: int(0), True: int(1)}, inplace=True)

#Sums up expenses for luxury attractions for the passenger
df["Expenses"]=df["RoomService"]+ df["FoodCourt"]+df["ShoppingMall"]+ df["Spa"]+ df["VRDeck"]

#I separate the number of the passenger group and information about his cabin
df[['Group','Number']] = df.PassengerId.str.split(pat='_', expand=True).astype(int)
df[['DeckC','NumberC', 'SideC']] = df.Cabin.str.split(pat='/', expand=True)



In [53]:
df=df[['PassengerId','HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP','Name','Expenses', 'Group', 'Number',
       'DeckC', 'NumberC', 'SideC', 'Transported' ]].copy()

#Looking for gender of passengers. 

df['name'] = df['Name'].str.split().str[0].str.upper()
df['lastCharName'] = df['Name'].str.split().str[0].str[-1]

# If last char passenger name == a then SeXW = 1 ( pessenger probably is women) else passenger should be men

df['SexW'] = np.where(np.logical_or(df['name'].isin(fNames.values) ,df2['lastCharName'] == 'a') , 1, 0)

impF = SimpleImputer(strategy="most_frequent")
df[['HomePlanet']] = impF.fit_transform(df[['HomePlanet']])
df[['CryoSleep']] = impF.fit_transform(df[['CryoSleep']])
df[['Destination']] = impF.fit_transform(df[['Destination']])
df[['VIP']] = impF.fit_transform(df[['VIP']])
df[['SideC']] = impF.fit_transform(df[['SideC']])
df[['DeckC']] = impF.fit_transform(df[['DeckC']])

impM = SimpleImputer(strategy="mean")
df[['Age']] = impM.fit_transform(df[['Age']])
df[['Expenses']] = impM.fit_transform(df[['Expenses']])
df[['NumberC']] = impM.fit_transform(df[['NumberC']])

df=df.drop(['Name', 'name', 'lastCharName'], axis=1)


In [56]:
#Handling with categorical data

ohe = OneHotEncoder(drop='first')

dfPlanet=ohe.fit_transform(df[['HomePlanet']])
df[ohe.get_feature_names_out()] = dfPlanet.toarray()

dfDestination=ohe.fit_transform(df[['Destination']])
df[ohe.get_feature_names_out()] = dfPlanet.toarray()

dfSideC=ohe.fit_transform(df[['SideC']])
df[ohe.get_feature_names_out()] = dfSideC.toarray()

dfDeckC=ohe.fit_transform(df[['DeckC']])
df[ohe.get_feature_names_out()] = dfDeckC.toarray()

shuffle(df, random_state=42)

prep_data=df.drop(columns=['PassengerId','DeckC', 'HomePlanet', 'Destination', 'SideC'], axis=1)


prep_data.head()
prep_data.to_csv('Data/train_preprocessed.csv', index=False)